In [0]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from google.colab import drive
import sys, os, shutil
from pathlib import Path
drive.mount("/content/drive", force_remount=True)
colabase = Path('/content/drive/My Drive/Colab Notebooks/')
course = Path('/content/drive/My Drive/course-v3/')
dlff = Path('/content/drive/My Drive/deeplearning-ff/')
#sys.path.append(str(dlff)) # cd {dlff}
os.chdir(dlff)

Mounted at /content/drive


In [0]:
#export
from exports.lg_11 import *

## Serializing the model

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=2920)

In [0]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [0]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=8)

In [5]:
len(il)

12954

In [0]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [0]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [0]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [10]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.089409,0.272443,2.022308,0.306000,00:41
1,1.896748,0.371286,1.911743,0.358000,00:42
2,1.761955,0.431026,1.796361,0.406000,00:42
3,1.679199,0.472700,1.789904,0.424000,00:42
4,1.614183,0.507066,1.751447,0.434000,00:42
5,1.564515,0.529629,1.787914,0.410000,00:42
6,1.525561,0.548338,1.969582,0.398000,00:42
7,1.475642,0.575719,1.671021,0.486000,00:42
8,1.417974,0.601092,1.645622,0.518000,00:42
9,1.393609,0.615465,2.087687,0.370000,00:42


In [0]:
st = learn.model.state_dict()

In [12]:
type(st)

collections.OrderedDict

In [13]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [14]:
st['10.bias']

tensor([ 0.0351, -0.0089, -0.0418, -0.0112,  0.0010,  0.0112,  0.0080,  0.0378,
         0.0066, -0.0497], device='cuda:0')

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [0]:
torch.save(st, mdl_path/'iw5')

## Pets

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3127)

In [0]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [18]:
pets.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [0]:
pets_path = pets/'images'

In [0]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [21]:
il

ImageList (7390 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_177.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_53.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_160.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_27.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_31.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_118.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Persian_159.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_24.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Ragdoll_89.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terrier_181.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images

In [0]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [0]:
random.seed(42)

In [0]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [25]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_177.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_160.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_27.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_31.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_118.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Persian_159.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Ragdoll_89.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pug_25.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/beagle_34.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Russian_Blue_192.jpg')...]
Path: /root/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_53.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/american_pit_bull_terri

In [26]:
n = il.items[0].name; n

'samoyed_177.jpg'

In [27]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'samoyed'

In [0]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [0]:
proc = CategoryProcessor()

In [0]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [31]:
', '.join(proc.vocab)

'samoyed, beagle, scottish_terrier, Sphynx, Persian, Ragdoll, pug, Russian_Blue, Bengal, Abyssinian, american_bulldog, shiba_inu, pomeranian, Maine_Coon, saint_bernard, wheaten_terrier, boxer, keeshond, Bombay, english_setter, german_shorthaired, yorkshire_terrier, Egyptian_Mau, english_cocker_spaniel, japanese_chin, great_pyrenees, havanese, leonberger, miniature_pinscher, newfoundland, basset_hound, British_Shorthair, chihuahua, Siamese, staffordshire_bull_terrier, Birman, american_pit_bull_terrier'

In [0]:
ll.valid.x.tfms = val_tfms

In [0]:
c_out = len(proc.vocab)

In [0]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [36]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.476174,0.084596,3.638457,0.082988,00:41
1,3.311148,0.128244,3.621751,0.112033,00:41
2,3.059625,0.200090,3.274661,0.134163,00:41
3,2.732168,0.287836,2.788664,0.271093,00:41
4,2.426794,0.392980,2.394021,0.398340,00:41


## Custom head

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3265)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [0]:
st = torch.load(mdl_path/'iw5')

In [0]:
m = learn.model

In [40]:
m.load_state_dict(st)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [42]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
pred = m_cut(xb)

In [44]:
pred.shape

torch.Size([128, 512, 4, 4])

In [0]:
ni = pred.shape[1]

In [0]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [0]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [0]:
learn.model = m_new

In [49]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.863926,0.291735,2.143609,0.500692,00:41
1,1.950638,0.551972,2.175661,0.504841,00:41
2,1.719087,0.630268,1.775272,0.603043,00:41
3,1.484615,0.721464,1.455772,0.734440,00:41
4,1.289853,0.794960,1.354365,0.773167,00:41


## adapt_model and gradual unfreezing

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3483)

In [0]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [51]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [52]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
for p in learn.model[0].parameters(): p.requires_grad_(False) #freeze

In [54]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.755721,0.304785,2.387255,0.412172,00:37
1,2.381153,0.425979,2.302367,0.450899,00:37
2,2.038576,0.536673,1.964307,0.528354,00:37


In [0]:
for p in learn.model[0].parameters(): p.requires_grad_(True) #unfreeze

In [56]:
learn.fit(5, cbsched, reset_opt=True) # batch norm is the pb; dont freeze batchnorm 

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.833846,0.611819,1.846803,0.597510,00:41
1,1.731677,0.645268,1.936381,0.547718,00:41
2,1.642020,0.671516,1.728410,0.645920,00:41
3,1.452714,0.743663,1.501218,0.734440,00:40
4,1.269533,0.821959,1.410249,0.773167,00:41


## Batch norm transfer

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3567)

In [57]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f) # recursive 

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [0]:
apply_mod(learn.model, partial(set_grad, b=False)) # freeze except BN and linear

In [60]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.652406,0.335983,2.154346,0.481328,00:39
1,2.071247,0.512074,1.868395,0.597510,00:39
2,1.809044,0.607770,1.741437,0.626556,00:39


In [0]:
apply_mod(learn.model, partial(set_grad, b=True)) # unfreeze

In [62]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.692303,0.651867,1.718769,0.641770,00:40
1,1.644002,0.672266,1.930646,0.561549,00:40
2,1.612817,0.677366,1.635047,0.666667,00:40
3,1.434337,0.745313,1.497771,0.723375,00:41
4,1.267647,0.817609,1.381651,0.771784,00:40


Pytorch already has an `apply` method we can use:

In [0]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3799)

In [0]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [65]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [0]:
# u cud freeze by just setting lr to 0 on the approp. layers 
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [0]:
a,b = bn_splitter(learn.model)

In [0]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [69]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [0]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [71]:
cb_types.after_backward

'after_backward'

In [0]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [0]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [0]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [75]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [76]:
def _print_det(o): 
    """Print details of hypers in each param.group
    >>learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])
    """
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [77]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.491959,0.370631,2.372985,0.419087,00:41
1,2.216984,0.459727,2.125524,0.489627,00:41
2,1.946059,0.559172,1.834496,0.609959,00:41


In [0]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [79]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.788373,0.615569,1.806225,0.604426,00:41
1,1.787957,0.607320,1.814463,0.598894,00:41
2,1.676174,0.649168,1.704918,0.647303,00:41
3,1.501177,0.718764,1.569500,0.708160,00:41
4,1.414571,0.754762,1.525504,0.712310,00:41


## Export

In [81]:
! python notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exports/lg_11a.py
